# Lactate Response
## Team 3 - Vital and Laboratory, First Six Hours
### MIT Critical Data

The purpose of this notebook is to summarise the vital and laboratory data for the first six hours in the ICU.

## 0 - Envrionment

In [1]:
import numpy as np
import pandas as pd
import psycopg2

dbname = 'mimic'
schema_name = 'mimiciii'
db_schema = 'SET search_path TO {0};'.format(schema_name)
con = psycopg2.connect(database=dbname)

## 1 - Vitals

In [2]:
query = db_schema + '''
SELECT pvt.subject_id, pvt.hadm_id, pvt.icustay_id

-- Easier names
, MIN(CASE WHEN VitalID = 1 THEN valuenum ELSE null END) AS HeartRate_MIN
, MAX(CASE WHEN VitalID = 1 THEN valuenum ELSE null END) AS HeartRate_MAX
, AVG(CASE WHEN VitalID = 1 THEN valuenum ELSE null END) AS HeartRate_Mean
, MIN(CASE WHEN VitalID = 2 THEN valuenum ELSE null END) AS SysBP_MIN
, MAX(CASE WHEN VitalID = 2 THEN valuenum ELSE null END) AS SysBP_MAX
, AVG(CASE WHEN VitalID = 2 THEN valuenum ELSE null END) AS SysBP_Mean
, MIN(CASE WHEN VitalID = 3 THEN valuenum ELSE null END) AS DiasBP_MIN
, MAX(CASE WHEN VitalID = 3 THEN valuenum ELSE null END) AS DiasBP_MAX
, AVG(CASE WHEN VitalID = 3 THEN valuenum ELSE null END) AS DiasBP_Mean
, MIN(CASE WHEN VitalID = 4 THEN valuenum ELSE null END) AS MeanBP_MIN
, MAX(CASE WHEN VitalID = 4 THEN valuenum ELSE null END) AS MeanBP_MAX
, AVG(CASE WHEN VitalID = 4 THEN valuenum ELSE null END) AS MeanBP_Mean
, MIN(CASE WHEN VitalID = 5 THEN valuenum ELSE null END) AS RespRate_MIN
, MAX(CASE WHEN VitalID = 5 THEN valuenum ELSE null END) AS RespRate_MAX
, AVG(CASE WHEN VitalID = 5 THEN valuenum ELSE null END) AS RespRate_Mean
, MIN(CASE WHEN VitalID = 6 THEN valuenum ELSE null END) AS TempC_MIN
, MAX(CASE WHEN VitalID = 6 THEN valuenum ELSE null END) AS TempC_MAX
, AVG(CASE WHEN VitalID = 6 THEN valuenum ELSE null END) AS TempC_Mean
, MIN(CASE WHEN VitalID = 7 THEN valuenum ELSE null END) AS SpO2_MIN
, MAX(CASE WHEN VitalID = 7 THEN valuenum ELSE null END) AS SpO2_MAX
, AVG(CASE WHEN VitalID = 7 THEN valuenum ELSE null END) AS SpO2_Mean
, MIN(CASE WHEN VitalID = 8 THEN valuenum ELSE null END) AS Glucose_MIN
, MAX(CASE WHEN VitalID = 8 THEN valuenum ELSE null END) AS Glucose_MAX
, AVG(CASE WHEN VitalID = 8 THEN valuenum ELSE null END) AS Glucose_Mean

FROM  (
  SELECT ie.subject_id, ie.hadm_id, ie.icustay_id
  , CASE
    WHEN itemid IN (211,220045) AND valuenum > 0 AND valuenum < 300 THEN 1 -- HeartRate
    WHEN itemid IN (51,442,455,6701,220179,220050) AND valuenum > 0 AND valuenum < 400 THEN 2 -- SysBP
    WHEN itemid IN (8368,8440,8441,8555,220180,220051) AND valuenum > 0 AND valuenum < 300 THEN 3 -- DiasBP
    WHEN itemid IN (456,52,6702,443,220052,220181,225312) AND valuenum > 0 AND valuenum < 300 THEN 4 -- MeanBP
    WHEN itemid IN (615,618,220210,224690) AND valuenum > 0 AND valuenum < 70 THEN 5 -- RespRate
    WHEN itemid IN (223761,678) AND valuenum > 70 AND valuenum < 120  THEN 6 -- TempF, converted to degC in valuenum call
    WHEN itemid IN (223762,676) AND valuenum > 10 AND valuenum < 50  THEN 6 -- TempC
    WHEN itemid IN (646,220277) AND valuenum > 0 AND valuenum <= 100 THEN 7 -- SpO2
    WHEN itemid IN (807,811,1529,3745,3744,225664,220621,226537) AND valuenum > 0 THEN 8 -- Glucose

    else null end as VitalID
      -- convert F to C
  , CASE WHEN itemid in (223761,678) THEN (valuenum-32)/1.8 else valuenum end as valuenum

  FROM icustays ie
  LEFT JOIN chartevents ce
  on ie.subject_id = ce.subject_id AND ie.hadm_id = ce.hadm_id AND ie.icustay_id = ce.icustay_id
  AND ce.charttime BETWEEN (ie.intime - interval '2' hour) AND (ie.intime + interval '6' hour)
  -- exclude rows marked as error
  AND ce.error IS DISTINCT FROM 1
  WHERE ce.itemid in
  (
  -- HEART RATE
  211, --"Heart Rate"
  220045, --"Heart Rate"

  -- Systolic/diastolic

  51, --	Arterial BP [Systolic]
  442, --	Manual BP [Systolic]
  455, --	NBP [Systolic]
  6701, --	Arterial BP #2 [Systolic]
  220179, --	Non Invasive Blood Pressure systolic
  220050, --	Arterial Blood Pressure systolic

  8368, --	Arterial BP [Diastolic]
  8440, --	Manual BP [Diastolic]
  8441, --	NBP [Diastolic]
  8555, --	Arterial BP #2 [Diastolic]
  220180, --	Non Invasive Blood Pressure diastolic
  220051, --	Arterial Blood Pressure diastolic


  -- MEAN ARTERIAL PRESSURE
  456, --"NBP Mean"
  52, --"Arterial BP Mean"
  6702, --	Arterial BP Mean #2
  443, --	Manual BP Mean(calc)
  220052, --"Arterial Blood Pressure mean"
  220181, --"Non Invasive Blood Pressure mean"
  225312, --"ART BP mean"

  -- RESPIRATORY RATE
  618,--	Respiratory Rate
  615,--	Resp Rate (Total)
  220210,--	Respiratory Rate
  224690, --	Respiratory Rate (Total)


  -- SPO2, peripheral
  646, 220277,

  -- GLUCOSE, both lab AND fingerstick
  807,--	Fingerstick Glucose
  811,--	Glucose (70-105)
  1529,--	Glucose
  3745,--	BloodGlucose
  3744,--	Blood GlucoseAND ce.charttime BETWEEN (ie.intime - interval '2' hour) AND (ie.intime + interval '6' hour)
  225664,--	Glucose finger stick
  220621,--	Glucose (serum)
  226537,--	Glucose (whole blood)

  -- TEMPERATURE
  223762, -- "Temperature Celsius"
  676,	-- "Temperature C"
  223761, -- "Temperature Fahrenheit"
  678 --	"Temperature F"

  )
) pvt
GROUP BY pvt.subject_id, pvt.hadm_id, pvt.icustay_id
ORDER BY pvt.subject_id, pvt.hadm_id, pvt.icustay_id;
'''

vitals_df = pd.read_sql_query(query, con)
vitals_df.head()

,subject_id,hadm_id,icustay_id,heartrate_min,heartrate_max,heartrate_mean,sysbp_min,sysbp_max,sysbp_mean,diasbp_min,...,resprate_mean,tempc_min,tempc_max,tempc_mean,spo2_min,spo2_max,spo2_mean,glucose_min,glucose_max,glucose_mean
0,2,163353,243653,131.0,148.0,140.750000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72.0,72.0,72.000000
1,3,145834,211552,95.0,168.0,138.555556,62.0,217.0,94.814815,24.0,...,16.545455,36.055556,36.777776,36.344444,74.0,100.0,95.642857,140.0,299.0,235.666667
2,4,185777,294638,74.0,111.0,89.166667,101.0,116.0,106.000000,57.0,...,NaN,37.444445,37.444445,37.444445,95.0,100.0,98.333333,172.0,183.0,179.333333
3,5,178980,214757,140.0,140.0,140.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,93.0,93.0,93.000000
4,6,107064,228232,79.0,87.0,82.625000,125.0,161.0,141.875000,56.0,...,13.125000,36.000002,36.555557,36.240742,99.0,100.0,99.750000,181.0,254.0,205.333333


## 2 - Labs

In [3]:
query = '''
SELECT
  pvt.subject_id, pvt.hadm_id, pvt.icustay_id

  , MIN(CASE WHEN label = 'ANION GAP' THEN valuenum ELSE null END) as ANIONGAP_MIN
  , MAX(CASE WHEN label = 'ANION GAP' THEN valuenum ELSE null END) as ANIONGAP_MAX
  , MIN(CASE WHEN label = 'ALBUMIN' THEN valuenum ELSE null END) as ALBUMIN_MIN
  , MAX(CASE WHEN label = 'ALBUMIN' THEN valuenum ELSE null END) as ALBUMIN_MAX
  , MIN(CASE WHEN label = 'BANDS' THEN valuenum ELSE null END) as BANDS_MIN
  , MAX(CASE WHEN label = 'BANDS' THEN valuenum ELSE null END) as BANDS_MAX
  , MIN(CASE WHEN label = 'BICARBONATE' THEN valuenum ELSE null END) as BICARBONATE_MIN
  , MAX(CASE WHEN label = 'BICARBONATE' THEN valuenum ELSE null END) as BICARBONATE_MAX
  , MIN(CASE WHEN label = 'BILIRUBIN' THEN valuenum ELSE null END) as BILIRUBIN_MIN
  , MAX(CASE WHEN label = 'BILIRUBIN' THEN valuenum ELSE null END) as BILIRUBIN_MAX
  , MIN(CASE WHEN label = 'CREATININE' THEN valuenum ELSE null END) as CREATININE_MIN
  , MAX(CASE WHEN label = 'CREATININE' THEN valuenum ELSE null END) as CREATININE_MAX
  , MIN(CASE WHEN label = 'CHLORIDE' THEN valuenum ELSE null END) as CHLORIDE_MIN
  , MAX(CASE WHEN label = 'CHLORIDE' THEN valuenum ELSE null END) as CHLORIDE_MAX
  , MIN(CASE WHEN label = 'GLUCOSE' THEN valuenum ELSE null END) as GLUCOSE_MIN
  , MAX(CASE WHEN label = 'GLUCOSE' THEN valuenum ELSE null END) as GLUCOSE_MAX
  , MIN(CASE WHEN label = 'HEMATOCRIT' THEN valuenum ELSE null END) as HEMATOCRIT_MIN
  , MAX(CASE WHEN label = 'HEMATOCRIT' THEN valuenum ELSE null END) as HEMATOCRIT_MAX
  , MIN(CASE WHEN label = 'HEMOGLOBIN' THEN valuenum ELSE null END) as HEMOGLOBIN_MIN
  , MAX(CASE WHEN label = 'HEMOGLOBIN' THEN valuenum ELSE null END) as HEMOGLOBIN_MAX
  , MIN(CASE WHEN label = 'LACTATE' THEN valuenum ELSE null END) as LACTATE_MIN
  , MAX(CASE WHEN label = 'LACTATE' THEN valuenum ELSE null END) as LACTATE_MAX
  , MIN(CASE WHEN label = 'PLATELET' THEN valuenum ELSE null END) as PLATELET_MIN
  , MAX(CASE WHEN label = 'PLATELET' THEN valuenum ELSE null END) as PLATELET_MAX
  , MIN(CASE WHEN label = 'POTASSIUM' THEN valuenum ELSE null END) as POTASSIUM_MIN
  , MAX(CASE WHEN label = 'POTASSIUM' THEN valuenum ELSE null END) as POTASSIUM_MAX
  , MIN(CASE WHEN label = 'PTT' THEN valuenum ELSE null END) as PTT_MIN
  , MAX(CASE WHEN label = 'PTT' THEN valuenum ELSE null END) as PTT_MAX
  , MIN(CASE WHEN label = 'INR' THEN valuenum ELSE null END) as INR_MIN
  , MAX(CASE WHEN label = 'INR' THEN valuenum ELSE null END) as INR_MAX
  , MIN(CASE WHEN label = 'PT' THEN valuenum ELSE null END) as PT_MIN
  , MAX(CASE WHEN label = 'PT' THEN valuenum ELSE null END) as PT_MAX
  , MIN(CASE WHEN label = 'SODIUM' THEN valuenum ELSE null END) as SODIUM_MIN
  , MAX(CASE WHEN label = 'SODIUM' THEN valuenum ELSE null end) as SODIUM_MAX
  , MIN(CASE WHEN label = 'BUN' THEN valuenum ELSE null end) as BUN_MIN
  , MAX(CASE WHEN label = 'BUN' THEN valuenum ELSE null end) as BUN_MAX
  , MIN(CASE WHEN label = 'WBC' THEN valuenum ELSE null end) as WBC_MIN
  , MAX(CASE WHEN label = 'WBC' THEN valuenum ELSE null end) as WBC_MAX


FROM
( -- begin query that extracts the data
  SELECT ie.subject_id, ie.hadm_id, ie.icustay_id
  -- here we assign labels to ITEMIDs
  -- this also fuses together multiple ITEMIDs containing the same data
  , CASE
        WHEN itemid = 50868 THEN 'ANION GAP'
        WHEN itemid = 50862 THEN 'ALBUMIN'
        WHEN itemid = 51144 THEN 'BANDS'
        WHEN itemid = 50882 THEN 'BICARBONATE'
        WHEN itemid = 50885 THEN 'BILIRUBIN'
        WHEN itemid = 50912 THEN 'CREATININE'
        WHEN itemid = 50806 THEN 'CHLORIDE'
        WHEN itemid = 50902 THEN 'CHLORIDE'
        WHEN itemid = 50809 THEN 'GLUCOSE'
        WHEN itemid = 50931 THEN 'GLUCOSE'
        WHEN itemid = 50810 THEN 'HEMATOCRIT'
        WHEN itemid = 51221 THEN 'HEMATOCRIT'
        WHEN itemid = 50811 THEN 'HEMOGLOBIN'
        WHEN itemid = 51222 THEN 'HEMOGLOBIN'
        WHEN itemid = 50813 THEN 'LACTATE'
        WHEN itemid = 51265 THEN 'PLATELET'
        WHEN itemid = 50822 THEN 'POTASSIUM'
        WHEN itemid = 50971 THEN 'POTASSIUM'
        WHEN itemid = 51275 THEN 'PTT'
        WHEN itemid = 51237 THEN 'INR'
        WHEN itemid = 51274 THEN 'PT'
        WHEN itemid = 50824 THEN 'SODIUM'
        WHEN itemid = 50983 THEN 'SODIUM'
        WHEN itemid = 51006 THEN 'BUN'
        WHEN itemid = 51300 THEN 'WBC'
        WHEN itemid = 51301 THEN 'WBC'
      ELSE null
    END AS label
  , -- add in some sanity checks on the values
  -- the where clause below requires all valuenum to be > 0, so these are only upper limit checks
    CASE
      WHEN itemid = 50862 and valuenum >    10 THEN null -- g/dL 'ALBUMIN'
      WHEN itemid = 50868 and valuenum > 10000 THEN null -- mEq/L 'ANION GAP'
      WHEN itemid = 51144 and valuenum <     0 THEN null -- immature band forms, %
      WHEN itemid = 51144 and valuenum >   100 THEN null -- immature band forms, %
      WHEN itemid = 50882 and valuenum > 10000 THEN null -- mEq/L 'BICARBONATE'
      WHEN itemid = 50885 and valuenum >   150 THEN null -- mg/dL 'BILIRUBIN'
      WHEN itemid = 50806 and valuenum > 10000 THEN null -- mEq/L 'CHLORIDE'
      WHEN itemid = 50902 and valuenum > 10000 THEN null -- mEq/L 'CHLORIDE'
      WHEN itemid = 50912 and valuenum >   150 THEN null -- mg/dL 'CREATININE'
      WHEN itemid = 50809 and valuenum > 10000 THEN null -- mg/dL 'GLUCOSE'
      WHEN itemid = 50931 and valuenum > 10000 THEN null -- mg/dL 'GLUCOSE'
      WHEN itemid = 50810 and valuenum >   100 THEN null -- % 'HEMATOCRIT'
      WHEN itemid = 51221 and valuenum >   100 THEN null -- % 'HEMATOCRIT'
      WHEN itemid = 50811 and valuenum >    50 THEN null -- g/dL 'HEMOGLOBIN'
      WHEN itemid = 51222 and valuenum >    50 THEN null -- g/dL 'HEMOGLOBIN'
      WHEN itemid = 50813 and valuenum >    50 THEN null -- mmol/L 'LACTATE'
      WHEN itemid = 51265 and valuenum > 10000 THEN null -- K/uL 'PLATELET'
      WHEN itemid = 50822 and valuenum >    30 THEN null -- mEq/L 'POTASSIUM'
      WHEN itemid = 50971 and valuenum >    30 THEN null -- mEq/L 'POTASSIUM'
      WHEN itemid = 51275 and valuenum >   150 THEN null -- sec 'PTT'
      WHEN itemid = 51237 and valuenum >    50 THEN null -- 'INR'
      WHEN itemid = 51274 and valuenum >   150 THEN null -- sec 'PT'
      WHEN itemid = 50824 and valuenum >   200 THEN null -- mEq/L == mmol/L 'SODIUM'
      WHEN itemid = 50983 and valuenum >   200 THEN null -- mEq/L == mmol/L 'SODIUM'
      WHEN itemid = 51006 and valuenum >   300 THEN null -- 'BUN'
      WHEN itemid = 51300 and valuenum >  1000 THEN null -- 'WBC'
      WHEN itemid = 51301 and valuenum >  1000 THEN null -- 'WBC'
    ELSE le.valuenum
    END AS valuenum

  FROM icustays ie

  LEFT JOIN labevents le
    ON le.subject_id = ie.subject_id AND le.hadm_id = ie.hadm_id
    AND le.charttime BETWEEN (ie.intime - interval '2' hour) AND (ie.intime + interval '6' hour)
    AND le.ITEMID in
    (
      -- comment is: LABEL | CATEGORY | FLUID | NUMBER OF ROWS IN LABEVENTS
      50868, -- ANION GAP | CHEMISTRY | BLOOD | 769895
      50862, -- ALBUMIN | CHEMISTRY | BLOOD | 146697
      51144, -- BANDS - hematology
      50882, -- BICARBONATE | CHEMISTRY | BLOOD | 780733
      50885, -- BILIRUBIN, TOTAL | CHEMISTRY | BLOOD | 238277
      50912, -- CREATININE | CHEMISTRY | BLOOD | 797476
      50902, -- CHLORIDE | CHEMISTRY | BLOOD | 795568
      50806, -- CHLORIDE, WHOLE BLOOD | BLOOD GAS | BLOOD | 48187
      50931, -- GLUCOSE | CHEMISTRY | BLOOD | 748981
      50809, -- GLUCOSE | BLOOD GAS | BLOOD | 196734
      51221, -- HEMATOCRIT | HEMATOLOGY | BLOOD | 881846
      50810, -- HEMATOCRIT, CALCULATED | BLOOD GAS | BLOOD | 89715
      51222, -- HEMOGLOBIN | HEMATOLOGY | BLOOD | 752523
      50811, -- HEMOGLOBIN | BLOOD GAS | BLOOD | 89712
      50813, -- LACTATE | BLOOD GAS | BLOOD | 187124
      51265, -- PLATELET COUNT | HEMATOLOGY | BLOOD | 778444
      50971, -- POTASSIUM | CHEMISTRY | BLOOD | 845825
      50822, -- POTASSIUM, WHOLE BLOOD | BLOOD GAS | BLOOD | 192946
      51275, -- PTT | HEMATOLOGY | BLOOD | 474937
      51237, -- INR(PT) | HEMATOLOGY | BLOOD | 471183
      51274, -- PT | HEMATOLOGY | BLOOD | 469090
      50983, -- SODIUM | CHEMISTRY | BLOOD | 808489
      50824, -- SODIUM, WHOLE BLOOD | BLOOD GAS | BLOOD | 71503
      51006, -- UREA NITROGEN | CHEMISTRY | BLOOD | 791925
      51301, -- WHITE BLOOD CELLS | HEMATOLOGY | BLOOD | 753301
      51300  -- WBC COUNT | HEMATOLOGY | BLOOD | 2371
    )
    AND valuenum IS NOT null AND valuenum > 0 -- lab values cannot be 0 and cannot be negative
) pvt
GROUP BY pvt.subject_id, pvt.hadm_id, pvt.icustay_id
ORDER BY pvt.subject_id, pvt.hadm_id, pvt.icustay_id;
'''

labs_df = pd.read_sql_query(query, con)
labs_df.head()

,subject_id,hadm_id,icustay_id,aniongap_min,aniongap_max,albumin_min,albumin_max,bands_min,bands_max,bicarbonate_min,...,inr_min,inr_max,pt_min,pt_max,sodium_min,sodium_max,bun_min,bun_max,wbc_min,wbc_max
0,2,163353,243653,NaN,NaN,NaN,NaN,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.1,22.0
1,3,145834,211552,22.0,23.0,1.8,1.8,NaN,NaN,11.0,...,1.7,1.7,15.6,15.7,138.0,153.0,41.0,43.0,11.3,19.1
2,4,185777,294638,15.0,15.0,2.8,2.8,NaN,NaN,21.0,...,1.1,1.1,12.8,12.8,141.0,141.0,10.0,10.0,9.7,9.7
3,5,178980,214757,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.9,13.9
4,6,107064,228232,20.0,20.0,NaN,NaN,NaN,NaN,18.0,...,1.0,1.1,12.6,12.6,134.0,134.0,65.0,65.0,10.6,10.6


## 3 - Merge and Save

In [4]:
vlsf6h_df = vitals_df.merge(labs_df, how='outer', on=['icustay_id', 'hadm_id', 'subject_id'])
print(vlsf6h_df.shape)
vlsf6h_df.head()

(61532, 65)


,subject_id,hadm_id,icustay_id,heartrate_min,heartrate_max,heartrate_mean,sysbp_min,sysbp_max,sysbp_mean,diasbp_min,...,inr_min,inr_max,pt_min,pt_max,sodium_min,sodium_max,bun_min,bun_max,wbc_min,wbc_max
0,2,163353,243653,131.0,148.0,140.750000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.1,22.0
1,3,145834,211552,95.0,168.0,138.555556,62.0,217.0,94.814815,24.0,...,1.7,1.7,15.6,15.7,138.0,153.0,41.0,43.0,11.3,19.1
2,4,185777,294638,74.0,111.0,89.166667,101.0,116.0,106.000000,57.0,...,1.1,1.1,12.8,12.8,141.0,141.0,10.0,10.0,9.7,9.7
3,5,178980,214757,140.0,140.0,140.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.9,13.9
4,6,107064,228232,79.0,87.0,82.625000,125.0,161.0,141.875000,56.0,...,1.0,1.1,12.6,12.6,134.0,134.0,65.0,65.0,10.6,10.6


In [5]:
vlsf6h_df.to_csv('~/Desktop/vlsf6h.csv', index=False)